In [ ]:
# import standard scientific libraries
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as kb

# import ML models from scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error

2023-10-30 07:40:25.863577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 07:40:26.074011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-30 07:40:26.074072: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-30 07:40:26.107557: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 07:40:27.582995: W tensorflow/stream_executor/platform/de

In [ ]:
pd.set_option('max_columns', None)
pd.set_option("display.precision", 8)

dataset = "./"

In [ ]:
 train = pd.read_csv(dataset + "train.csv")
train.shape

(68613, 14)

In [ ]:
col = ["functional_groups", "topology"]
for i in col:
    train[i] = train[i].astype("category").cat.codes
train.dtypes

MOFname                                           object
volume [A^3]                                     float64
weight [u]                                       float64
surface_area [m^2/g]                             float64
void_fraction                                    float64
void_volume [cm^3/g]                             float64
functional_groups                                  int16
metal_linker                                       int64
organic_linker1                                    int64
organic_linker2                                    int64
topology                                            int8
CO2/N2_selectivity                               float64
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]    float64
CO2_working_capacity [mL/g]                      float64
dtype: object

In [ ]:
ratio = 0.8
train_size = math.floor(train.shape[0]*0.8)
test_size = train.shape[0] - train_size
print("dataset size:", train.shape[0])
print("train size:", train_size)
print("test size:", test_size)

dataset size: 68613
train size: 54890
test size: 13723


In [ ]:
train = train.iloc[:, [1,2,3,4,5,7,8,9,11,12,13]]
train

,volume [A^3],weight [u],surface_area [m^2/g],void_fraction,void_volume [cm^3/g],metal_linker,organic_linker1,organic_linker2,CO2/N2_selectivity,heat_adsorption_CO2_P0.15bar_T298K [kcal/mol],CO2_working_capacity [mL/g]
0,1116.667429,875.240600,0.00,0.07899,0.0607,3,4,11,22.86416611,6.786041,105.28450172
1,2769.503842,2211.697211,603.61,0.13794,0.1040,10,44,57,33.61678033,7.147286,101.22477418
2,1089.818728,773.687960,788.50,0.14874,0.1262,2,22,24,19.26372569,6.347967,118.98701075
3,2205.198301,1304.638720,1441.53,0.21814,0.2220,9,17,24,25.70137692,6.190085,187.62600450
4,1137.800963,901.736120,0.00,0.07778,0.0591,2,1,22,30.00183795,6.478063,79.21000066
...,...,...,...,...,...,...,...,...,...,...,...
68608,1188.302573,1001.700216,0.00,0.00000,0.0000,3,4,24,24.13177036,NaN,-12.94365213
68609,1506.660363,1493.296496,0.00,0.01108,0.0000,10,42,46,6.07181778,NaN,-12.98558157
68610,2035.532738,1959.518320,0.00,0.00000,0.0000,4,14,22,9.87613400,NaN,-13.18763527
68611,3985.426053,3638.677280,0.00,0.00000,0.0000,4,4,15,5.28505063,inf,15.67269762


In [ ]:
x_train = train.iloc[0:train_size, :-1].astype(np.float32)
y_train = train.iloc[0:train_size, -1].astype(np.float32)
x_test = train.iloc[train_size:train_size+test_size, :-1].astype(np.float32)
y_true = train.iloc[train_size:train_size+test_size, -1].astype(np.float32)
x_train.dtypes

volume [A^3]                                     float32
weight [u]                                       float32
surface_area [m^2/g]                             float32
void_fraction                                    float32
void_volume [cm^3/g]                             float32
metal_linker                                     float32
organic_linker1                                  float32
organic_linker2                                  float32
CO2/N2_selectivity                               float32
heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]    float32
dtype: object

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
def custom_loss(y_actual, y_pred): 
    custom_loss=kb.log(kb.sum(kb.abs(y_actual - y_pred)) / y_actual.shape[0])
    return custom_loss

In [ ]:
# กำหนด sample data
x = np.random.randint(1, 4, size=(1000,))
x = np.asarray(x).T

y = x ** 2
y = np.asarray(y).T
# แปลงข้อมูลให้เป็น float32
## ถ้าสงสัยว่าแปลงทำไม ลองไปหาอ่านเพิ่มดูนะ :P
x = x.astype(np.float32)
y = y.astype(np.float32)

# สร้าง neural net
keras_model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation=tf.nn.relu, input_shape=[1]),
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(1),
    ]
)

# กำหนด optimizer
optimizer = tf.keras.optimizers.RMSprop(0.001)

# สร้างโมเดล
keras_model.compile(loss=custom_loss, optimizer=optimizer)

# เทรนโมเดล
keras_model.fit(x, y, batch_size=20, epochs=50)

Epoch 1/50
2023-10-30 07:40:30.403177: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-30 07:40:30.403212: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-30 07:40:30.403230: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-89ff0356-6e0b-40ab-922e-602a46fe5c41): /proc/driver/nvidia/version does not exist
2023-10-30 07:40:30.403485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
50/50 [==============================] - 1s 1ms/step - loss: 1.2286
Epoch

In [ ]:
print(keras_model.predict([11]))

1/1 [==============================] - 0s 106ms/step
[[45.1263]]


In [ ]:
tf.random.set_seed(42)

keras_model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation=tf.nn.relu, input_shape=[1]),
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(1),
    ]
)

optimizer = tf.keras.optimizers.RMSprop(0.001)

keras_model.compile(loss=custom_loss, optimizer=optimizer)

keras_model.fit(x_train, y_train, batch_size=20, epochs=50)

Epoch 1/50


ValueError: in user code:

    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/shared-libs/python3.7/py/lib/python3.7/site-packages/keras/engine/input_spec.py", line 278, in assert_input_

In [ ]:
# Set random seed
tf.random.set_seed(42)

# Create a model (same as above)
model = tf.keras.Sequential([
  tf.keras.layers.Dense(1),
  #tf.keras.layers.Dense(1)
])

# Compile model (same as above)
model.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["mae"])

# Fit model (this time we'll train for longer)
model.fit(x_train, y_train, epochs=10) # train for 100 epochs not 10

Epoch 1/10
1716/1716 [==============================] - 2s 990us/step - loss: 55898.4772 - mae: 55898.4772
Epoch 2/10
1716/1716 [==============================] - 2s 986us/step - loss: 55311.5233 - mae: 55311.5233
Epoch 3/10
1716/1716 [==============================] - 2s 933us/step - loss: 57199.5665 - mae: 57199.5665
Epoch 4/10
1716/1716 [==============================] - 2s 991us/step - loss: 56286.0352 - mae: 56286.0352
Epoch 5/10
1716/1716 [==============================] - 2s 1ms/step - loss: 55550.4433 - mae: 55550.4433
Epoch 6/10
1716/1716 [==============================] - 2s 1ms/step - loss: 55868.7599 - mae: 55868.7599
Epoch 7/10
1716/1716 [==============================] - 2s 1ms/step - loss: 56649.6699 - mae: 56649.6699
Epoch 8/10
1716/1716 [==============================] - 2s 1ms/step - loss: 56239.6440 - mae: 56239.6440
Epoch 9/10
1716/1716 [==============================] - 2s 964us/step - loss: 56818.7611 - mae: 56818.7611
Epoch 10/10
1716/1716 [======================

In [ ]:
# Set random seed
tf.random.set_seed(42)

# Replicate model_2
model_3 = tf.keras.Sequential([
  tf.keras.layers.Dense(1),
  tf.keras.layers.Dense(1)
])

# Compile the model
model_3.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.SGD(),
                metrics=['mae'])

# Fit the model (this time for 500 epochs, not 100)
model_3.fit(x_train, y_train, epochs=10, verbose=1) # set verbose to 0 for less output

Epoch 1/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 2/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 3/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 4/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 5/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 6/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 7/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 8/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 9/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan
Epoch 10/10
1716/1716 [==============================] - 2s 1ms/step - loss: nan - mae: nan


In [ ]:
# Set random seed
tf.random.set_seed(42)

# Replicate original model
model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(1)
])

# Compile the model
model_1.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.SGD(),
                metrics=['mae'])

# Fit the model
model_1.fit(x_train, y_train, epochs=100)

Epoch 1/100
1716/1716 [==============================] - 2s 1ms/step - loss: 55898.4772 - mae: 55898.4772
Epoch 2/100
1716/1716 [==============================] - 2s 1ms/step - loss: 55311.5233 - mae: 55311.5233
Epoch 3/100
1716/1716 [==============================] - 2s 1ms/step - loss: 57199.5665 - mae: 57199.5665
Epoch 4/100
1716/1716 [==============================] - 2s 1ms/step - loss: 56286.0352 - mae: 56286.0352
Epoch 5/100
1716/1716 [==============================] - 2s 1ms/step - loss: 55550.4433 - mae: 55550.4433
Epoch 6/100
1716/1716 [==============================] - 2s 1ms/step - loss: 55868.7599 - mae: 55868.7599
Epoch 7/100
1716/1716 [==============================] - 2s 1ms/step - loss: 56649.6699 - mae: 56649.6699
Epoch 8/100
1716/1716 [==============================] - 2s 1ms/step - loss: 56239.6440 - mae: 56239.6440
Epoch 9/100
1716/1716 [==============================] - 2s 1ms/step - loss: 56818.7611 - mae: 56818.7611
Epoch 10/100
1716/1716 [======================

In [ ]:
model2 = Sequential()
model2.add(Dense(12, input_dim=(x_train.shape[1]), activation='relu')) # input
model2.add(Dense(24, activation='relu')) # hidden 1
model2.add(Dense(12, activation='relu')) # hidden 2
model2.add(Dense(1, activation='linear')) # output

In [ ]:
model2.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
model2.fit(x_train, y_train, epochs=30, batch_size=64)

Epoch 1/30
858/858 [==============================] - 1s 1ms/step - loss: 119.9927 - accuracy: 0.0000e+00
Epoch 2/30
858/858 [==============================] - 1s 1ms/step - loss: 44.2328 - accuracy: 0.0000e+00
Epoch 3/30
858/858 [==============================] - 1s 1ms/step - loss: 38.4332 - accuracy: 0.0000e+00
Epoch 4/30
858/858 [==============================] - 1s 1ms/step - loss: 35.8902 - accuracy: 0.0000e+00
Epoch 5/30
858/858 [==============================] - 1s 1ms/step - loss: 35.4641 - accuracy: 0.0000e+00
Epoch 6/30
858/858 [==============================] - 1s 1ms/step - loss: 34.5384 - accuracy: 0.0000e+00
Epoch 7/30
858/858 [==============================] - 1s 1ms/step - loss: 34.4160 - accuracy: 0.0000e+00
Epoch 8/30
858/858 [==============================] - 1s 1ms/step - loss: 33.9488 - accuracy: 0.0000e+00
Epoch 9/30
858/858 [==============================] - 1s 1ms/step - loss: 34.1204 - accuracy: 0.0000e+00
Epoch 10/30
858/858 [==============================] -

In [ ]:
y_pred = model2.predict(x_test)
#log_mae = np.log(mean_absolute_error(y_pred, y_true))
#log_mae

In [ ]:
y_pred



array([[111.84691],
       [274.0271 ],
       [165.82558],
       ...,
       [      nan],
       [      nan],
       [      nan]], dtype=float32)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=89ff0356-6e0b-40ab-922e-602a46fe5c41' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>